In [1]:
import numpy as np
import numpy.linalg as la
import pandas as pd
from sklearn.linear_model import Lasso
from scipy.stats import norm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

# autoreload
%load_ext autoreload
%autoreload 2

# import data and code
from Project_2 import *
print(f'The data contains {dat.shape[0]} rows (countries) and {dat.shape[1]} columns (variables).')

The data contains 214 rows (countries) and 85 columns (variables).


**Single Lasso using BRT**

In [2]:
''' Step 1: Calculate BRT penalty '''
penalty_BRT = BRT(X_tilde, g)

''' Step 2: Lasso g using y0 and Z (i.e. g using X) '''
fit_BRTgx, coeff_BRTgx, intercept_BRTgx = lasso(X_tilde, g, penalty_BRT)

# Implied estimates and selection of non-zero coefficient variables for further analysis
Z_J_BRTgx = Z[:,coeff_BRTgx[1:]!=0] # Note: We use Z and not Z_tilde
selected_variables_BRTgx = (coeff_BRTgx != 0)

# Display number of variables in Z_J
if Z_J_BRTgx.size > 0: # if Z_J is non-empty and variables have been selected
    print("The number of variables in Z_J is {}".format(Z_J_BRTgx.shape[1]))
    print('Selected varriables: ', X_names[selected_variables_BRTgx].to_list())
else: # if Z_J is empty
    print("The number of variables in Z_J is 0, no variables selected.")

''' Step 3: Regress g using y0 and Z_J (selected variables) using OLS '''
# Add a constant to X
xx = np.column_stack((np.ones(N),y0,Z_J_BRTgx))
yy = np.array(g).reshape(-1,1)

# Calculate OLS estimate
coefs_BRT_SPL = la.inv(xx.T@xx)@xx.T@yy
beta_y0_BRTSPL = coefs_BRT_SPL[1][0]

#print the coefficients
pd.DataFrame(coefs_BRT_SPL, index=['constant', 'lgdp_initial'], columns=['gdp_growth'])

''' Step 4: Calculate variance, standard errors and confidence interval '''
# Estimate variance
var_BRT_SPL = var_single(xx,yy,coefs_BRT_SPL)

# Calculate standard errors
se_BRT_SPL = standard_errors1(var_BRT_SPL)
print("se_BRT_PSL = ",se_BRT_SPL.round(5))

# Calculate confidence interval
CI_low_BRT_SPL  = beta_y0_BRTSPL - q * se_BRT_SPL
CI_high_BRT_SPL = beta_y0_BRTSPL + q * se_BRT_SPL

# Display confidence interval
print("Confidence interval for β_y0 (BRT_SPL) = ",(CI_low_BRT_SPL.round(5),CI_high_BRT_SPL.round(5)))

NameError: name 'X_tilde' is not defined